In [ ]:
import os
import pickle
import pandas as pd
from inference_weather import download_weather_for_inference

# Load data for inference

In [26]:
def extract_year_month(filename):
    parts = filename.replace(".csv", "").split("_")
    month = int(parts[1])
    year = int(parts[2])
    return year, month

In [ ]:
try:
    csv_files = [f for f in os.listdir('../data/weather_daily') if f.endswith(".csv")]

    latest_file = max(csv_files, key=lambda f: extract_year_month(f))
    
    weather_features = pd.read_csv('../data/weather_daily/' + latest_file)

except:
    weather_features = download_weather_for_inference()

try:
    ndvi_features = pd.read_csv('../data/ndvi/ndvi_final.csv')
    ndvi_features['Month'] = pd.to_datetime(ndvi_features['Month'])
    
except:
    pass # TODO: RUN data_ndvi.ipynb

print(f"Datapoints (weather): {len(weather_features)}")
print(weather_features.columns)

print(f"Datapoints (NDVI): {len(ndvi_features)}")
print(ndvi_features.columns)


Datapoints (weather): 13209
Index(['Lat', 'Lon', 'ssrd_lag1', 'ssrd_lag2', 'ssrd_lag3', 'swvl1_lag1',
       'swvl1_lag2', 'swvl1_lag3', 'swvl2_lag1', 'swvl2_lag2', 'swvl2_lag3',
       't2m_lag1', 't2m_lag2', 't2m_lag3', 'tp_lag1', 'tp_lag2', 'tp_lag3',
       'valid_time'],
      dtype='object')
Datapoints (NDVI): 5022
Index(['row_id', 'Lat', 'Lon', 'Month', 'NDVI', 'NDVI_clim', 'NDVI_anom',
       'NDVI_lag1', 'NDVI_anom_lag1', 'NDVI_lag2', 'NDVI_anom_lag2'],
      dtype='object')


# Load Model

In [ ]:
model_files = next(os.walk('../models'), (None, None, []))[2]  
file_name = '../models/' + model_files[-1]
model = pickle.load(open(file_name, "rb")) # [-1] to get the latest version
